# Lab 1: Three Basic HR Agents

**Learning Objectives:**
- Build Agent 1: No tools, No memory (Basic chatbot)
- Build Agent 2: With tools, No memory (Data lookup)
- Build Agent 3: With tools, With memory (Full conversational agent)

**Time:** 30 minutes

## Setup: Install Dependencies

In [2]:
!pip install --pre -U langchain langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.6/484.6 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.3.34
    Uninstalling langchain-openai-0.3.34:
      Successfully uninstalled langchain-openai-0.3.34
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 0.6.4
    Uninstalling langgraph-prebuilt-0.6.4:
      Successfully uninstalled langgraph-prebuilt-0.6.4
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.6.8
    Uninstalling l

## Setup: Configure OpenAI API Key

In [3]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
# Set OPENAI_API_KEY as an ENV
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

---
# Agent 1: WITHOUT Tools and WITHOUT Memory

**Features:**
- ❌ No tools
- ❌ No memory
- ✅ Can answer general questions using LLM knowledge

In [5]:
from langchain.agents import create_agent

# Create Agent 1: No tools, No memory
agent_1 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],  # Empty list = No tools!
    prompt="You are an HR assistant. You can chat and answer general HR questions.",
)

print("✅ Agent 1 created: NO tools, NO memory")
print("="*60)

✅ Agent 1 created: NO tools, NO memory


### Test Agent 1

In [6]:
# Test Agent 1 with a general HR question
result = agent_1.invoke({
    "messages": [{"role": "user", "content": "What are common HR policies?"}]
})

print("Question: What are common HR policies?")
print("\nAgent 1 Response:")
print(result['messages'][-1].content)

Question: What are common HR policies?

Agent 1 Response:
Common HR policies include:

1. **Equal Employment Opportunity (EEO) Policy**: Ensures a workplace free from discrimination based on race, gender, age, religion, disability, or other protected characteristics.

2. **Harassment Policy**: Outlines unacceptable behavior, such as sexual harassment, and procedures for reporting incidents.

3. **Code of Conduct**: Sets expectations for employee behavior, including professionalism, integrity, and compliance with laws.

4. **Attendance and Punctuality Policy**: Details expectations for work hours, reporting absences, and consequences for violations.

5. **Leave of Absence Policy**: Covers types of leave available (e.g., sick leave, vacation, FMLA) and procedures for requesting leave.

6. **Performance Review Policy**: Establishes a process for evaluating employee performance, including frequency and criteria.

7. **Compensation and Benefits Policy**: Outlines how salaries, bonuses, and 

In [7]:
# Try a follow-up question - NO MEMORY, so won't remember context
result = agent_1.invoke({
    "messages": [{"role": "user", "content": "Can you give me 3 examples?"}]
})

print("Question: Can you give me 3 examples?")
print("\nAgent 1 Response:")
print(result['messages'][-1].content)
print("\n⚠️ Notice: Agent doesn't know what examples to give (no memory of previous question)")

Question: Can you give me 3 examples?

Agent 1 Response:
Sure! Could you please specify what type of examples you are looking for? For instance, are you interested in examples related to HR policies, employee engagement strategies, interview questions, or something else?

⚠️ Notice: Agent doesn't know what examples to give (no memory of previous question)


---
# Define Tools (for Agent 2 and 3)

We'll create two simple tools:
1. **get_employee_info** - Look up employee details
2. **check_leave_balance** - Check leave days remaining

In [8]:
# Tool 1: Get Employee Information
def get_employee_info(employee_id: str) -> str:
    """Get employee information by ID."""
    employees = {
        "101": "Priya Sharma - Engineering - Senior Developer",
        "102": "Rahul Verma - Engineering - Manager",
        "103": "Anjali Patel - HR - Director",
        "104": "Arjun Reddy - Sales - Team Lead",
        "105": "Sneha Gupta - Marketing - Specialist"
    }
    return employees.get(employee_id, f"Employee {employee_id} not found")

# Tool 2: Check Leave Balance
def check_leave_balance(employee_id: str) -> str:
    """Check remaining leave days for an employee by ID."""
    leave_data = {
        "101": "Priya Sharma has 12 days of leave remaining",
        "102": "Rahul Verma has 8 days of leave remaining",
        "103": "Anjali Patel has 15 days of leave remaining",
        "104": "Arjun Reddy has 10 days of leave remaining",
        "105": "Sneha Gupta has 5 days of leave remaining"
    }
    return leave_data.get(employee_id, f"Leave data for employee {employee_id} not found")

print("✅ Tools defined:")
print("   1. get_employee_info - Look up employee details")
print("   2. check_leave_balance - Check leave balance")
print("\n📋 Employee Database:")
print("   101: Priya Sharma")
print("   102: Rahul Verma")
print("   103: Anjali Patel")
print("   104: Arjun Reddy")
print("   105: Sneha Gupta")

✅ Tools defined:
   1. get_employee_info - Look up employee details
   2. check_leave_balance - Check leave balance

📋 Employee Database:
   101: Priya Sharma
   102: Rahul Verma
   103: Anjali Patel
   104: Arjun Reddy
   105: Sneha Gupta


---
# Agent 2: WITH Tools but WITHOUT Memory

**Features:**
- ✅ Has tools (can access employee data)
- ❌ No memory (each question is independent)
- ✅ Good for one-time lookups

In [9]:
# Create Agent 2: With tools, No memory
agent_2 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance],  # Has tools!
    prompt="You are an HR assistant. Use tools to help users find employee information and check leave balances.",
    # No checkpointer = No memory!
)

print("✅ Agent 2 created: HAS tools, NO memory")
print("="*60)

✅ Agent 2 created: HAS tools, NO memory


### Test Agent 2 - Tool Usage

In [10]:
# Test: Ask about employee info
result = agent_2.invoke({
    "messages": [{"role": "user", "content": "Who is employee 101?"}]
})

print("Question: Who is employee 101?")
print("\nAgent 2 Response:")
print(result['messages'][-1].content)
print("\n✅ Agent used get_employee_info tool!")

Question: Who is employee 101?

Agent 2 Response:
Employee 101 is Priya Sharma, who works in the Engineering department as a Senior Developer.

✅ Agent used get_employee_info tool!


In [11]:
# Test: Ask about leave balance
result = agent_2.invoke({
    "messages": [{"role": "user", "content": "How many leave days does employee 102 have?"}]
})

print("Question: How many leave days does employee 102 have?")
print("\nAgent 2 Response:")
print(result['messages'][-1].content)
print("\n✅ Agent used check_leave_balance tool!")

Question: How many leave days does employee 102 have?

Agent 2 Response:
Employee 102, Rahul Verma, has 8 days of leave remaining.

✅ Agent used check_leave_balance tool!


### Test Agent 2 - No Memory Problem

In [13]:
# Try a follow-up question - NO MEMORY
result = agent_2.invoke({
    "messages": [{"role": "user", "content": "What department is she in?"}]
})

print("Question: What department is she in?")
print("\nAgent 2 Response:")
print(result['messages'][-1].content)
print("\n⚠️ Problem: Agent doesn't know who 'she' refers to (no memory!)")

Question: What department is she in?

Agent 2 Response:
I can help you with that! Please provide the employee's ID so I can retrieve the information.

⚠️ Problem: Agent doesn't know who 'she' refers to (no memory!)


---
# Agent 3: WITH Tools AND WITH Memory

**Features:**
- ✅ Has tools (can access employee data)
- ✅ Has memory (remembers conversation)
- ✅ Can handle follow-up questions
- ✅ Best for real conversations

In [14]:
from langgraph.checkpoint.memory import InMemorySaver

# Create memory
checkpointer = InMemorySaver()

# Create Agent 3: With tools AND with memory
agent_3 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance],  # Has tools!
    prompt="You are an HR assistant. Use tools to help users find employee information and check leave balances.",
    checkpointer=checkpointer  # Has memory!
)

print("✅ Agent 3 created: HAS tools, HAS memory")
print("="*60)

✅ Agent 3 created: HAS tools, HAS memory


### Test Agent 3 - With Memory

In [15]:
# Start a conversation with thread_id
config = {"configurable": {"thread_id": "conversation_1"}}

# First question
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "Who is employee 101?"}]},
    config
)

print("Question 1: Who is employee 101?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)

Question 1: Who is employee 101?

Agent 3 Response:
Employee 101 is Priya Sharma, who works in the Engineering department as a Senior Developer.


In [16]:
# Follow-up question - MEMORY WORKS!
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "What's her leave balance?"}]},
    config  # Same thread_id
)

print("Question 2: What's her leave balance?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)
print("\n✅ Success: Agent remembers we're talking about Priya Sharma (employee 101)!")

Question 2: What's her leave balance?

Agent 3 Response:
Priya Sharma has 12 days of leave remaining.

✅ Success: Agent remembers we're talking about Priya Sharma (employee 101)!


In [17]:
# Another follow-up
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "What department is she in?"}]},
    config
)

print("Question 3: What department is she in?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)
print("\n✅ Success: Agent still remembers the context!")

Question 3: What department is she in?

Agent 3 Response:
Priya Sharma is in the Engineering department.

✅ Success: Agent still remembers the context!


### Test Agent 3 - Complex Memory

In [18]:
# Ask about another employee
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "Now tell me about employee 102"}]},
    config
)

print("Question 4: Now tell me about employee 102")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)

Question 4: Now tell me about employee 102

Agent 3 Response:
Employee 102 is Rahul Verma, who works in the Engineering department as a Manager.


In [19]:
# Ask to compare - needs memory of BOTH employees!
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "Who has more leave days, the first employee or this one?"}]},
    config
)

print("Question 5: Who has more leave days, the first employee or this one?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)
print("\n✅ Amazing: Agent remembers BOTH employee 101 (Priya) AND 102 (Rahul)!")

Question 5: Who has more leave days, the first employee or this one?

Agent 3 Response:
Priya Sharma (employee 101) has 12 days of leave remaining, while Rahul Verma (employee 102) has 8 days of leave remaining. Therefore, Priya Sharma has more leave days.

✅ Amazing: Agent remembers BOTH employee 101 (Priya) AND 102 (Rahul)!


---
# Comparison: All Three Agents

## Summary Table

| Feature | Agent 1 | Agent 2 | Agent 3 |
|---------|---------|---------|----------|
| **Tools** | ❌ No | ✅ Yes | ✅ Yes |
| **Memory** | ❌ No | ❌ No | ✅ Yes |
| **Access employee data** | ❌ No | ✅ Yes | ✅ Yes |
| **Remember conversation** | ❌ No | ❌ No | ✅ Yes |
| **Handle follow-ups** | ❌ No | ❌ No | ✅ Yes |
| **Use case** | General chat | One-time lookups | Full HR assistant |

## Side-by-Side Test

In [21]:
test_question = "Tell me about employee 103"

print("="*80)
print(f"Testing all agents with: '{test_question}'")
print("="*80)

# Agent 1
print("\n🤖 Agent 1 (No tools, No memory):")
print("-" * 60)
result1 = agent_1.invoke({
    "messages": [{"role": "user", "content": test_question}]
})
print(result1['messages'][-1].content)

# Agent 2
print("\n🤖 Agent 2 (With tools, No memory):")
print("-" * 60)
result2 = agent_2.invoke({
    "messages": [{"role": "user", "content": test_question}]
})
print(result2['messages'][-1].content)

# Agent 3
print("\n🤖 Agent 3 (With tools, With memory):")
print("-" * 60)
test_config = {"configurable": {"thread_id": "test_thread"}}
result3 = agent_3.invoke(
    {"messages": [{"role": "user", "content": test_question}]},
    test_config
)
print(result3['messages'][-1].content)

print("\n" + "="*80)

Testing all agents with: 'Tell me about employee 103'

🤖 Agent 1 (No tools, No memory):
------------------------------------------------------------
I'm sorry, but I can't provide specific information about employees, including employee 103, due to privacy and confidentiality protocols. However, I can help answer general HR questions or provide information on topics like employee benefits, performance management, recruitment, and more. How can I assist you today?

🤖 Agent 2 (With tools, No memory):
------------------------------------------------------------
Employee 103 is Anjali Patel, who holds the position of Director in the HR department.

🤖 Agent 3 (With tools, With memory):
------------------------------------------------------------
Employee 103 is Anjali Patel, who serves as the Director in the HR department.



## Follow-up Test (Only Agent 3 can handle this!)

In [22]:
followup_question = "What's their leave balance?"

print("="*80)
print(f"Follow-up question: '{followup_question}'")
print("="*80)

print("\n🤖 Agent 3 (With tools, With memory):")
print("-" * 60)
result3_followup = agent_3.invoke(
    {"messages": [{"role": "user", "content": followup_question}]},
    test_config  # Same thread!
)
print(result3_followup['messages'][-1].content)

print("\n✅ Only Agent 3 can handle this follow-up because it has MEMORY!")
print("   It remembers we were talking about employee 103 (Anjali Patel)")
print("\n" + "="*80)

Follow-up question: 'What's their leave balance?'

🤖 Agent 3 (With tools, With memory):
------------------------------------------------------------
Anjali Patel has 15 days of leave remaining.

✅ Only Agent 3 can handle this follow-up because it has MEMORY!
   It remembers we were talking about employee 103 (Anjali Patel)



---
# Try It Yourself!

Now it's your turn! Try asking your own questions to each agent.

In [ ]:
# Customize this cell with your own questions!

my_question = "Who is employee 104?"  # Change this to your question
my_followup = "What about their leave?"  # Change this to your follow-up

# Test with Agent 3 (best one)
my_config = {"configurable": {"thread_id": "my_conversation"}}

print(f"Your question: {my_question}")
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": my_question}]},
    my_config
)
print(f"\nResponse: {result['messages'][-1].content}")

print(f"\n\nYour follow-up: {my_followup}")
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": my_followup}]},
    my_config
)
print(f"\nResponse: {result['messages'][-1].content}")

---
# Conclusion

**What you learned:**
1. ✅ How to create a basic agent (no tools, no memory)
2. ✅ How to add tools to an agent (for data access)
3. ✅ How to add memory to an agent (for conversations)
4. ✅ The difference between each approach

**Key Takeaways:**
- **Tools** = Access to data/functions
- **Memory** = Remember conversation context
- **Best practice**: Start simple, add features as needed

**Next Steps:**
- Try adding more tools (e.g., update_leave_balance)
- Connect to a real database instead of hardcoded data
- Add more complex conversation flows
- Deploy as a web service

---
**Created with:** LangChain + OpenAI + LangGraph